# Set up

## Chrome driver install & set up

In [ ]:
# 이 부분은 처음 한번만 실행하면 됨.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver
!sudo /usr/lib/chromium-browser/chromedriver /usr/bin ## added

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.EhusJrRrF5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.cEsYFnmb52/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.4o1eK9ZBGg/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

## Import library

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import os
import urllib.request
from urllib.request import Request, urlopen
from urllib.parse import quote_plus
import chardet
import math

## Mount Google Drive & Set Directory

In [ ]:
from google.colab import drive
drive.mount( '')

ValueError: ignored

In [ ]:
directory = 'G:\My Drive\00_projects\03_motherhood_topicmodeling/'


# Scrap

### Open Chrome driver

In [ ]:
service = Service(executable_path="/usr/bin/chromedriver") # changed
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


### 로그인

In [ ]:
driver.get('https://www.chosun.com/subscribe/signin/')
time.sleep(5)

# 아이디 입력
user = driver.find_element(By.ID, 'username')
user.send_keys('lhjren92@gmail.com')

# 비밀번호 입력
pswd = driver.find_element(By.ID, 'subsPassword')
pswd.send_keys('hl2363@NG427')

# 로그인 클릭
sign = driver.find_element(By.ID, 'subsSignIn')
sign.click()

### 조선 뉴스 라이브러리 아카이브



In [ ]:
for kwd in ['양육']:

    # 빈 데이터프레임 만들기 - 날짜, 지면(사회, 정치), 제목, 본문
    df = pd.DataFrame(columns=['날짜', '지면', '종류', '제목', 'sub1', 'sub2', 'sub3', 'sub4'])

    # 뉴스 라이브러리 검색 페이지로 이동
    driver.get('https://newslibrary.chosun.com/search/search.html')
    time.sleep(5)

    # search query 입력
    search = driver.find_element(By.ID, 'query')
    search.send_keys(kwd)
    time.sleep(2)

    # 검색기간 설정 클릭
    filter = driver.find_element(By.XPATH, "//label[@for='date_select']")
    filter.click()
    time.sleep(2)

    # 검색기간 입력
    ds = driver.find_element(By.ID, "ds")
    ds.send_keys("19200101")

    de = driver.find_element(By.ID, "de")
    de.send_keys("19260108")

    #time.sleep(2)

    # 검색 클릭
    searchbutton = driver.find_element(By.XPATH, "//*[@id='set_box_id']/div[2]/button")
    searchbutton.click()

    # 검색결과 건수 retrieve
    searchN = int(driver.find_element(By.XPATH, '//*[@id="total"]').text.strip('건').replace(',',''))

    # scrap
    for i in range(0, math.floor(searchN/10)+1):

        # 기사목록 불러오기
        all_links = driver.find_elements(By.ID, 'result_list_kwd')[0].find_elements(By.TAG_NAME, 'dt')

        for j in range(0, len(all_links)):
            # 기사로 넘어가기 전에 제목 scrap
            title = all_links[j].find_element(By.TAG_NAME, 'a').text                      # 제목

            # 기사로 넘어가기
            all_links[j].find_element(By.TAG_NAME, 'a').click()
            WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reporter_info"]/li[3]')))

            # 기사로 넘어간 후에 내용 scrap
            date =  driver.find_element(By.XPATH, '//*[@id="reporter_info"]/li[1]').text   # 날짜
            page = driver.find_element(By.XPATH, '//*[@id="reporter_info"]/li[2]').text   # 지면
            cat = driver.find_element(By.XPATH, '//*[@id="reporter_info"]/li[3]').text    # 종류
            sub1 = driver.find_element(By.XPATH, '//*[@id="subtitle1"]').text             # sub1
            sub2 = driver.find_element(By.XPATH, '//*[@id="subtitle2"]').text             # sub2
            sub3 = driver.find_element(By.XPATH, '//*[@id="subtitle3"]').text             # sub3
            sub4 = driver.find_element(By.XPATH, '//*[@id="subtitle4"]').text             # sub4

            # 데이터프레임에 저장
            df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},
                        ignore_index = True)

            # check
            print(i*10 + j + 1, title, date)

            # 검색 결과로 돌아가기
            driver.back()
            WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="paginate_m_kwd"]/button[2]')))
            #time.sleep(10)

            # set the element again to avoid stale-element
            all_links = driver.find_elements(By.ID, 'result_list_kwd')[0].find_elements(By.TAG_NAME, 'dt')

        # 다음 페이지 버튼 누르기
        if i*10 + j + 1 < searchN:
            driver.find_element(By.XPATH, '//*[@id="paginate_m_kwd"]/button[2]').click()
            time.sleep(5)
        else:
            pass

    df.to_excel(directory + 'IMF_' + kwd + '_조선뉴스라이브러리.xlsx', index = False)

<ipython-input-22-4a18a14dbaaa>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},


1 兩處(양처)에棄兒(기아) 1926.01.08


<ipython-input-22-4a18a14dbaaa>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},


2 박명한 구세아 1926.01.08


<ipython-input-22-4a18a14dbaaa>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},


...

326 위생 시설 완전에 대하여 1920.07.17


<ipython-input-22-4a18a14dbaaa>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},


327 어린 아이에게 제일모친주의 1920.07.02


<ipython-input-22-4a18a14dbaaa>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'날짜':date, '지면':page, '종류':cat, '제목':title, 'sub1':sub1, 'sub2':sub2, 'sub3':sub3, 'sub4':sub4},


328 천외쇄언(天外瑣言) 1920.06.09


In [ ]:
df.to_excel(directory + 'IMF_' + kwd + "19260108" +'_조선뉴스라이브러리.xlsx', index = False)